First, functions and tools:
Summary: Using Tools with LangChain
Overview
This demo shows how to create and integrate tools into LangChain workflows. Tools allow the LLM to delegate specific tasks to external functions by invoking them during the conversation, enabling a powerful method for function calling within AI applications.

Key Steps Covered
1. Tool Creation
Tools are simple Python functions decorated with @tool.
A good description must be provided so the LLM can understand when to call it.
@tool("multiply")
def multiply(x: int, y: int) -> int:
  """Multiply two numbers together."""
  return x * y
The tool is registered into a tools list and a tool map for easy lookup by name.
tools = [multiply]
tool_map = {tool.name: tool for tool in tools}
2. Binding Tools to an LLM
Tools are bound to the LLM using bind_tools().
llm_with_tools = llm.bind_tools(tools)
This makes the LLM aware that it can call tools during a conversation if needed.
3. Invoking with Tools
A typical message list includes:
A SystemMessage setting the assistant's behavior.
A HumanMessage containing the user query (e.g., "What is 3 multiplied by 2?").
messages = [
  SystemMessage(content="You are a helpful assistant."),
  HumanMessage(content="What is 3 multiplied by 2?")
]
Invoking the LLM:
If the LLM identifies a tool to call, it produces tool_calls instead of a direct text response.
Otherwise, it produces regular content.
response = llm_with_tools.invoke(messages)
Example:
"How are you?" → Direct response, no tool_calls.
"What is 3 multiplied by 2?" → Empty content + tool_calls for the multiply function.
4. Handling Tool Calls Programmatically
Tool calls must be handled by the developer:
Parse tool_calls from the LLM response using LangChain utilities.
Extract the function name, arguments, and call ID.
Execute the corresponding tool using the arguments.
parsed_calls = parse_tool_calls(response.additional_kwargs["tool_calls"])
tool_call = parsed_calls[0]
function_name = tool_call["name"]
args = tool_call["args"]

tool = tool_map[function_name]
result = tool.invoke(**args)
After executing the tool:
A ToolMessage is created with the result, linking it back to the original tool_call ID.
tool_message = ToolMessage(content=str(result), tool_call_id=tool_call["id"])
messages.append(tool_message)
5. Sending the Result Back to the LLM
After the tool result is appended to the messages list:
A second invocation sends the updated conversation to the LLM.
The LLM then generates a final, natural language response incorporating the tool result.
final_response = llm_with_tools.invoke(messages)
Example final output: "3 multiplied by 2 is 6."
6. Summary of the Flow
Human asks a question.
LLM identifies if a tool is needed.
Tool is called manually by the application.
Tool output is fed back to the LLM.
LLM produces a complete answer.
7. Conclusion
Tools in LangChain enable structured, reliable, and expandable interaction patterns.
Developers must manage tool execution and message flow carefully.
This method paves the way for building powerful agentic systems that combine LLM reasoning with external capabilities.

Demo: Simple Agentic Workflows


In [ ]:
from typing import List, Dict
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AIMessage, 
    HumanMessage, 
    SystemMessage, 
    ToolMessage
)
from langchain_core.tools.structured import StructuredTool
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls
from dotenv import load_dotenv

In [ ]:
load_dotenv()
# if dotenv is set up

**The Agent Class**

In [ ]:
class Agent:
    def __init__(
            self, 
            name:str="AI Agent",
            role:str="Personal Assistant",
            instructions:str = "Help users with any question", 
            model:str="gpt-4o-mini",
            temperature:float=0.0,            
            tools:List[StructuredTool]=[]):
        
        self.name = name
        self.role = role
        self.instructions = instructions

        self.llm = ChatOpenAI(
            model=model,
            temperature=temperature,
        )

        self.tools = tools
        self.tool_map = {tool.name:tool for tool in tools}
        self.memory = [
            SystemMessage(
                content=f"You're {self.name}, your role is {self.role}, " 
                        f"and you need to {self.instructions} "
            ),
        ]
        
    def invoke(self, user_message:str):
        self.memory.append(HumanMessage(content=user_message))
        ai_message = self._invoke_llm()

        tool_calls = ai_message.additional_kwargs.get('tool_calls')
        if tool_calls:
            self._call_tools(tool_calls)
            self._invoke_llm()

        return self.memory[-1].content

    def _invoke_llm(self)->AIMessage:
        llm = self.llm.bind_tools(self.tools)
        ai_message = llm.invoke(self.memory)
        self.memory.append(ai_message)
        return ai_message

    def _call_tools(self, tool_calls:List[Dict]):
        parsed_tool_calls = parse_tool_calls(tool_calls)
        for tool_call in parsed_tool_calls:
            tool_call_id = tool_call['id']
            function_name = tool_call['name']
            arguments = tool_call['args']
            func = self.tool_map[function_name]
            result = func.invoke(arguments)
            tool_message = ToolMessage(
                content=result,
                name=function_name,
                tool_call_id=tool_call_id,
            )
            self.memory.append(tool_message)

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
agent = Agent(
    tools=[multiply]
)

In [ ]:
agent.invoke("2 multiplied by 2")

In [ ]:
agent.memory

Summary: Building a Basic Agent Abstraction in LangChain
Overview
This demo shows how to create a lightweight agent abstraction that layers together LLMs, memory, and tool use. The goal is to allow seamless handling of user inputs, tool calls, and model responses without having to manually orchestrate every piece at each step.

Key Steps Covered
1. Initial Setup
Standard imports and environment loading.
OpenAI chat model is instantiated as the LLM.
Tools are defined and organized:
A simple multiply tool is created.
A tool map is built for easy lookup by tool name.
@tool("multiply")
def multiply(x: int, y: int) -> int:
  """Multiply two numbers together."""
  return x * y

tools = [multiply]
tool_map = {tool.name: tool for tool in tools}
2. Memory Initialization
A Memory object is created to track the conversation history.
The memory is seeded with a SystemMessage to instruct the model on its role and behavior.
memory.add_message("system", "You are a helpful assistant specialized in math operations.")
3. Agent Class Design
An Agent class is implemented with the following key features:

Constructor
Takes parameters like:
name, role, instructions
model, temperature
tools (optional)
Sets up the LLM, tools, tool map, and initializes memory.
invoke() Method
Accepts a user_message input.
Appends the user's message to memory.
Invokes the LLM with the current memory.
Detects if the LLM produced a tool call:
If yes, it delegates the action to the call_tool() method.
Otherwise, it records the assistant's reply.
Returns the final assistant response from the latest memory entry.
def invoke(self, user_message):
  memory.add_message("user", user_message)
  ai_response = llm.invoke(memory.get_messages())
  if ai_response.tool_calls:
      self.call_tool(ai_response.tool_calls)
  else:
      memory.add_message("assistant", ai_response.content)
  return memory.last_message().content
call_tool() Method
Handles the parsing of tool calls.
Executes the appropriate tool based on the call and appends the result back into memory as a ToolMessage.
def call_tool(self, tool_calls):
  tool_call = tool_calls[0]
  function_name = tool_call.function.name
  args = json.loads(tool_call.function.arguments)
  tool = self.tool_map[function_name]
  result = tool.invoke(**args)
  memory.add_message("tool", str(result), tool_call_id=tool_call.id)
4. Execution Example
An agent is created using default parameters and the multiply tool.

User input: "What is 2 multiplied by 2?"

The agent flow:

User input is added to memory.
LLM recognizes the need to call the multiply tool.
Tool is called and the result (4) is captured.
Result is sent back to the LLM.
LLM responds naturally: "2 multiplied by 2 is 4."
Memory inspection shows:

System message
Human message
AI tool call
Tool message with result
Final AI message
5. Conclusion
This basic agent design creates a clean abstraction for:
Managing conversation history
Handling tool execution
Producing coherent LLM responses
It's a strong starting point for building more sophisticated, autonomous agent systems.